# Threshold Spreading

In [1]:
# Imports

import numpy as np
import networkx as nx

In [2]:
G = nx.read_edgelist('M1/edges.csv', delimiter = ',')
print(G)

Graph with 1429 nodes and 19357 edges


We first create a function which computes the new infected nodes using threshold reinforcement:

In [4]:
# Modified SI function using threshold
def new_infected_threshold(G, I, kappa, t):
    '''SI function which uses thresholds to determine which new 
    nodes are infected by their neighbours in one step of the infection process.
    
    Args:
        G (nx.Graph): Graph to perform algorithm on
        I (set): Set of already infected nodes
        kappa (int): Threshold for rest of spreading process
        t (int): Time step

    Return:
        new_infected (set): Set of newly infected nodes.
    '''

    # Create a new set of the newly infected nodes
    new_infected = set()

    # Check if time step t is 0 and set threshold to 1 if true
    if t == 0:
        kappa = 1

    # Iterate over all nodes in the network which are not already infected
    for u in set(G.nodes) - I:

        # Check if the amount of infected neighbors passes the threshold
        if len(set(G.neighbors(u)) & I) >= kappa:
            new_infected.add(u)

    return new_infected